In [103]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('/Users/prateekanand/Desktop/CS148/Travel-Purpose-Prediction/csv/trippub.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(data.head())

Preprocessing

In [ ]:
# Drop columns related to trip purpose, also drop HH_CBSA because most values are XXXXX and cannot impute those
drop = ['WHYTO', 'WHYFROM', 'TRIPPURP', 'WHYTRP1S', 'WHYTRP90', 'HH_CBSA']
y = data['WHYTRP1S']
X = data.drop(columns=drop, axis=1)

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(categorical_cols)

categorical_idx = [X.columns.get_loc(col) for col in categorical_cols]
print(categorical_idx)

print(X.head())

In [ ]:
# Invalid/missing values are reported as negative values in the dataframe or the string '-9'
X = X.map(lambda x: np.nan if ((isinstance(x, (int, float)) and x < 0) or (isinstance(x, str) and x == '-9')) else x)

# Replace these two columns with mode since they are categorical
mode = X['OBHUR'].mode()
X['OBHUR'].fillna(mode, inplace=True)
mode = X['DBHUR'].mode()
X['DBHUR'].fillna(mode, inplace=True)

print(X.head())

In [ ]:
# Then impute the numerical nan values with mean
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

for col in numerical_cols:
    mean = X[col].mean()
    X[col].fillna(mean, inplace=True)

print(X.head())

In [ ]:
# One-hot encode categorical features
X = pd.get_dummies(X, columns = categorical_cols)
X.replace({True: 1, False: 0}, inplace=True)

print(X.shape)
print(X.head())

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [111]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train, y_train)

/opt/homebrew/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [112]:
from sklearn.metrics import accuracy_score
y_pred = mlp.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.6692797011612485
